# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [15]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [16]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [17]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [208]:
data1 = data.groupby(["CustomerID","ProductName"]).agg(sum=("Quantity","sum"))
data1

sum
CustomerID ProductName                       
33         Apricots - Dried                 1
           Assorted Desserts                1
           Bandage - Flexible Neon          1
           Bar Mix - Pina Colada, 355 Ml    1
           Beans - Kidney, Canned           1
...                                       ...
98200      Vol Au Vents                    50
           Wasabi Powder                   25
           Wine - Fume Blanc Fetzer        25
           Wine - Hardys Bankside Shiraz   25
           Yogurt - French Vanilla         25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [209]:
pivotTable1 = pd.pivot_table(data, 
                            index= "ProductName", 
                            columns= "CustomerID", 
                            values= 'Quantity',
                            fill_value=0)
pivotTable=pivotTable1.transpose()
pivotTable

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0,0,0,25,0,25,0,25,0,0,...,0,25,25,0,0,0,0,0,0,0
98069,0,0,0,25,0,25,0,0,0,25,...,0,0,0,0,0,0,0,0,0,0
98159,0,0,0,0,0,0,0,0,0,0,...,0,25,0,0,0,0,25,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [210]:
dist_calculation = pdist(X=pivotTable, metric='euclidean')
dist_distribution = squareform(dist_calculation)

In [211]:
distance_eucl = pd.DataFrame(1/(1 + squareform(pdist(pivotTable, 'euclidean'))),
                         index=pivotTable.index, columns=pivotTable.index)

distance_eucl.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [212]:
similar_customer = distance_eucl[200].sort_values(ascending=False)[1:6]
similar_customer

CustomerID
1920    0.094414
1072    0.092610
3317    0.090094
3253    0.090094
3909    0.089695
Name: 200, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [230]:
records_customers = data1.loc[similar_customer.index]
records_customers

sum
CustomerID ProductName                        
1920       Apricots - Halves                 1
           Bacardi Breezer - Tropical        1
           Barramundi                        1
           Beans - Kidney White              1
           Beef - Texas Style Burger         1
...                                        ...
3909       Wine - Charddonnay Errazuriz      2
           Wine - Fume Blanc Fetzer          1
           Wine - Magnotta, Merlot Sr Vqa    1
           Wine - Toasted Head               1
           Wine - Valpolicella Masi          1

[261 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [234]:
aggregate_product = records_customers.groupby('ProductName').agg(sum_Quantity=('sum', 'sum')).sort_values(by='sum_Quantity', ascending=False)
aggregate_product

,sum_Quantity
ProductName,
Soup - Campbells Bean Medley,3
Lamb - Ground,3
Pomello,3
Bay Leaf,3
Pork - Kidney,3
...,...
Kiwi,1
Knife Plastic - White,1
"Lamb - Pieces, Diced",1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [215]:
df_merged = pd.merge(aggregate_product, pivotTable1, on = "ProductName")
costumer_table = df_merged.sort_values(by= 'Quantity', ascending = False)[[200,'Quantity']].head()
costumer_table

,200,Quantity
ProductName,,
Soup - Campbells Bean Medley,0,3
Pomello,0,3
Bay Leaf,0,3
Pork - Kidney,0,3
Muffin - Carrot Individual Wrap,0,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [227]:
recommendations = {}
customer_list = [100, 5000, 20000, 50000, 75000]

In [228]:
def product_recommendations(CostumerID):
    for i in costumer_list:
        similar_customer = distance_eucl[i].sort_values(ascending=False)[1:6]
        records_customers = data1.loc[similar_customer.index]
        aggregate_product = records_customers.groupby('ProductName').agg(sum_Quantity=('sum', 'sum')).sort_values(by='sum_Quantity', ascending=False)
        df_merged = pd.merge(pivotTable1, aggregate_product, on = "ProductName")
        costumer_table = df_merged.sort_values(by= 'sum_Quantity', ascending = False)[[i,'sum_Quantity']].head()
        recommendations[i] = costumer_table
        return recommendations


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
step_9 = pd.DataFrame({"customerName":step_8_dic_from_list.keys(),"recommendatedProducts":step_8_dic_from_list.values()})
step_9.head()

In [221]:
recommendations = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()


ValueError: Must pass 2-d input. shape=(1, 5, 2)

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [236]:
dist_calculation = pdist(X=pivotTable, metric='correlation')
dist_distribution = squareform(dist_calculation)

In [237]:
distance_eucl = pd.DataFrame(1/(1 + squareform(pdist(pivotTable, 'correlation'))),
                         index=pivotTable.index, columns=pivotTable.index)

distance_eucl.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493805,0.529875,0.515035,0.500570,0.504367,0.487132,0.492406,0.494838,0.500815,...,0.491005,0.486159,0.498339,0.480313,0.497157,0.480750,0.492063,0.485173,0.482914,0.496579
200,0.493805,1.000000,0.500545,0.496849,0.472695,0.483532,0.504988,0.481550,0.500545,0.506509,...,0.508587,0.496849,0.517060,0.506602,0.497813,0.487096,0.528445,0.499846,0.483420,0.484852
264,0.529875,0.500545,1.000000,0.498278,0.517080,0.493067,0.494101,0.481681,0.493067,0.489652,...,0.511597,0.493648,0.496434,0.502956,0.513787,0.526176,0.513045,0.506386,0.510648,0.494672
356,0.515035,0.496849,0.498278,1.000000,0.494727,0.489104,0.494727,0.510742,0.484642,0.490190,...,0.498600,0.489643,0.501690,0.498569,0.495876,0.497664,0.478039,0.507100,0.507023,0.499809
412,0.500570,0.472695,0.517080,0.494727,1.000000,0.494101,0.503354,0.510747,0.489748,0.504478,...,0.485869,0.513118,0.492877,0.499410,0.500310,0.502343,0.482847,0.497874,0.510144,0.522382
